In [ ]:
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [ ]:
path = "../data/Receptanteckningar_justerad.xlsx"
df = pd.read_excel(path, sheet_name=None)

Split data in antibiotics and no antibiotics class

In [ ]:
no_antibiotics = df['Ej AB 2018']
with pd.option_context('display.max_colwidth', None):
    display(no_antibiotics.head(5))

In [ ]:
antibiotics = pd.concat([df['AB 2012'], df['AB 2014'], df['AB 2017'], df['AB 2018']], axis=0).reset_index(drop=True)
with pd.option_context('display.max_colwidth', None):
    display(antibiotics.head(5))

### Removing all NAN cells

In [ ]:
nan_remove_noab = np.where(pd.isnull(no_antibiotics))[0].tolist()
no_antibiotics = no_antibiotics.drop(nan_remove_noab).reset_index(drop=True)

nan_remove_ab = np.where(pd.isnull(antibiotics))[0].tolist()
antibiotics = antibiotics.drop(nan_remove_ab).reset_index(drop=True)

### Remove cells starting with Elektroniskt recept

In [ ]:
def remove_electronic_recipe(dataframe):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Elektroniskt recept', regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    return remove_rows

print('No AB') # for no antibiotics
remove_rows_noab = remove_electronic_recipe(no_antibiotics)
no_antibiotics_new = no_antibiotics.drop(remove_rows_noab).reset_index(drop=True)

print('AB')    # for antibiotics
remove_rows_ab = remove_electronic_recipe(antibiotics)
antibiotics_new = antibiotics.drop(remove_rows_ab).reset_index(drop=True)

### Remove html tags

In [ ]:
def cleantext(dataframe):
    for i,t in enumerate(dataframe['Anteckning']):
        ct = BeautifulSoup(t, "lxml").text
        dataframe['Anteckning'][i] = ct
        
cleantext(no_antibiotics_new)
cleantext(antibiotics_new)

### Masking years

In [ ]:
def remove_year(dataframe, year):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(year, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(year, '[YEAR]', dataframe['Anteckning'][i])

for i in range(2011,2021,1):
    year = str(i)+'-'
    remove_year(no_antibiotics_new, year)
    remove_year(antibiotics_new, year)
    
for i in range(11,21,1):
    year = str(i)+'-'
    remove_year(no_antibiotics_new, year)
    remove_year(antibiotics_new, year)
    
for i in range(11,21,1):
    year = str(i)+'/'
    remove_year(no_antibiotics_new, year)   
    remove_year(antibiotics_new, year)

In [ ]:
antibiotics_new['Anteckning'][22124]

### Masking antibiotics from list

In [ ]:
antib = ["kåvepenin", "tikacillin", "avopen", "peceve", "dalacin", "clindamycin", "amimox", 
         "amoxicillin", "imacillin", "doxyferm", "doxycyklin", "vibranord", "abboticin", 
         "ery-max", "flagyl", "metronidazol", "primve", "metronidazol", "klindamycin",
        "Kåvepenin", "Tikacillin", "Avopen", "Peceve", "Dalacin", "Clindamycin", "Amimox", 
         "Amoxicillin", "Imacillin", "Doxyferm", "Doxycyklin", "Vibranord", "Abboticin", 
         "Ery-max", "Flagyl", "Metronidazol", "Primve", "Metronidazol", "Llindamycin", 
        "pc", "penicillin", "antibiotika", "ab", "Pc", "Penicillin", "Antibiotika", "Ab", 
        "PC", "AB"]

def remove_drug(dataframe, drug):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(drug, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    print(len(remove_y), drug)   
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(' '+drug+' ', ' [PAD] ', dataframe['Anteckning'][i])
        dataframe['Anteckning'][i] = re.sub('\A'+drug+' ', '[PAD] ', dataframe['Anteckning'][i])
        dataframe['Anteckning'][i] = re.sub(' '+drug+'\.', ' [PAD].', dataframe['Anteckning'][i])
        

for a in antib:
    remove_drug(no_antibiotics_new, a)

for a in antib:
    remove_drug(antibiotics_new, a)

In [ ]:
no_antibiotics_new['Anteckning'][3737]

### Remove rows starting with 'Osignerade'

In [ ]:
def remove_osi(dataframe):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Osignerade', regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    return remove_rows


remove_rows_noab = remove_osi(no_antibiotics_new)
no_antibiotics_new = no_antibiotics_new.drop(remove_rows_noab).reset_index(drop=True)

remove_rows_ab = remove_osi(antibiotics_new)
antibiotics_new = antibiotics_new.drop(remove_rows_ab).reset_index(drop=True)

### Remove rows containing skriver recept

In [ ]:
def remove_recipe2(dataframe):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Skriver recept', regex=True)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    return remove_rows


# For no antibiotics
remove_rows_noab = remove_recipe2(no_antibiotics_new)
for i in remove_rows_noab:
    sentences = no_antibiotics_new['Anteckning'][i].split('.')
    for s in sentences:
        if 'Skriver recept' in s:
            sentences.remove(s)
    no_antibiotics_new['Anteckning'][i] = ".".join(sentences)
    
# For no antibiotics
remove_rows_ab = remove_recipe2(antibiotics_new)
for i in remove_rows_ab:
    sentences = antibiotics_new['Anteckning'][i].split('.')
    for s in sentences:
        if 'Skriver recept' in s:
            sentences.remove(s)
    antibiotics_new['Anteckning'][i] = ".".join(sentences)

### Remove empty rows

In [ ]:
def remove_empty(dataframe):
    l = []
    for i,text in enumerate(dataframe['Anteckning']):
        if len(text) == 0:
            l.append(i)
    return l 


remove_rows_noab = remove_empty(no_antibiotics_new)
no_antibiotics_new = no_antibiotics_new.drop(remove_rows_noab).reset_index(drop=True)

remove_rows_ab = remove_empty(antibiotics_new)
antibiotics_new = antibiotics_new.drop(remove_rows_ab).reset_index(drop=True)

## Grouping cells by unikt nummer, datum and klinik for both classes

In [ ]:
with pd.option_context('display.max_colwidth', None):
    merge_no_antibiotics = no_antibiotics_new.groupby(['Unikt nummer','Datum','Klinik'])['Anteckning'].apply(lambda Anteckning: ''.join(Anteckning.to_string(index=False))).str.replace('(\\n)', ' ').reset_index()
    merge_antibiotics = antibiotics_new.groupby(['Unikt nummer','Datum','Klinik'])['Anteckning'].apply(lambda Anteckning: ''.join(Anteckning.to_string(index=False))).str.replace('(\\n)', ' ').reset_index()

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(merge_no_antibiotics.head(5))
    display(merge_antibiotics.head(5))

## Statistics No Antibiotics

In [ ]:
a = merge_no_antibiotics['Unikt nummer'].value_counts(sort=True).values
unique, counts = np.unique(a, return_counts=True)
l = len(a)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.xlabel('Number of visits')
plt.ylabel('Patients')
plt.savefig('visits_noab.png', bbox_inches='tight')
plt.show()

## Statistics Antibiotics

In [ ]:
a = merge_antibiotics['Unikt nummer'].value_counts(sort=True).values
idx = merge_antibiotics['Unikt nummer'].value_counts(sort=True).index
unique, counts = np.unique(a, return_counts=True)
l = len(a)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)
    
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.xlabel('Number of visits')
plt.ylabel('Patients')
plt.savefig('visits_ab.png', bbox_inches='tight')
plt.show()

mask = np.where(a>1,True,False)
visits = np.array([mask*idx])
visits = visits[visits!=0]
sep = []
for v in visits:
    dates = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].values
    for i in range(1,len(dates)):
        days = dates[i] - dates[i-1]
        sep.append(int(days.astype('timedelta64[D]')/ np.timedelta64(1, 'D')))
        
sep = np.array(sep)
month = sep[sep<15]
print('Percentage of second visits within 30 days', len(month)/len(sep))

unique, counts = np.unique(month, return_counts=True)
l = len(sep)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)
    
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.xlabel('Number of visits')
plt.ylabel('Patients')
plt.savefig('visits_ab_30.png', bbox_inches='tight')
plt.show()

# Joining visits within 14 days for AB class

In [ ]:
visits = np.array([mask*idx])
visits = visits[visits!=0]
for v in visits:
    dates = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].values
    for i in range(1,len(dates)):
        days = dates[i] - dates[i-1]
        days = int(days.astype('timedelta64[D]')/ np.timedelta64(1, 'D'))
        if days<15:
            ind = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].index[i]
            merge_antibiotics.at[ind, 'Datum'] = dates[i-1]
            dates[i] = dates[i-1]

### Remove spaces

In [ ]:
l = merge_no_antibiotics.shape[0]
for i in range(l):
    line = merge_no_antibiotics['Anteckning'][i]
    merge_no_antibiotics.at[i, 'Anteckning'] = " ".join(line.split())
    
l = merge_antibiotics.shape[0]
for i in range(l):
    line = merge_antibiotics['Anteckning'][i]
    merge_antibiotics.at[i, 'Anteckning'] = " ".join(line.split())

### Count of characters for AB class

In [ ]:
l = merge_antibiotics.shape[0]
length = []
maxn = 0
minn = 1000
idx_max = 0
idx_min = 0

for i in range(l):
    line = merge_antibiotics['Anteckning'][i]
    length.append(len(line))
    if len(line)>maxn:
        maxn = len(line)
        idx_max = i
    if len(line)<minn:
        minn = len(line)
        idx_min = i
        
    
print(np.mean(length))
print(min(length))
print(max(length))

print(maxn, minn)
print(idx_max, idx_min)

sns.distplot(length)
plt.xlabel('Characters count antibiotics');
plt.savefig('characters_count_ab.png', bbox_inches='tight')


### Count of characters for no AB class

In [ ]:
l = merge_no_antibiotics.shape[0]
length = []
maxn = 0
minn = 1000
idx_max = 0
idx_min = 0
for i in range(l):
    line = merge_no_antibiotics['Anteckning'][i]
    length.append(len(line))
    if len(line)>maxn:
        maxn = len(line)
        idx_max = i
    if len(line)<minn:
        minn = len(line)
        idx_min = i
        
    
print(np.mean(length))
print(min(length))
print(max(length))

print(maxn, minn)
print(idx_max, idx_min)

sns.distplot(length)
plt.xlabel('Char count no antibiotics');
plt.savefig('characters_count_noab.png', bbox_inches='tight')


### Saving each class separetely

In [ ]:
merge_antibiotics.to_csv('AB.csv')
merge_no:antibiotics.to_csv('Ej_AB.csv')

### Joining classes

In [ ]:
dataset={'Anteckning':[m for m in merge_antibiotics['Anteckning']],
         'Label': [1 for i in range(len(merge_antibiotics['Anteckning']))]}

for n in merge_no_antibiotics['Anteckning']:
    dataset['Anteckning'].append(n)
    dataset['Label'].append(0)
    
dataset_antibiotics = pd.DataFrame(dataset, columns = ['Anteckning', 'Label'])

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(dataset_antibiotics.head(5))

### Removing empty cells

In [ ]:
remove_rows = remove_empty(dataset_antibiotics)
dataset_antibiotics = dataset_antibiotics.drop(remove_rows).reset_index(drop=True)

### Removing names from dataset

In [ ]:
def remove_name(dataframe,name):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(name, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(' '+name+' ', ' [PAD] ', dataframe['Anteckning'][i])
        dataframe['Anteckning'][i] = re.sub('\A'+name+' ', '[PAD] ', dataframe['Anteckning'][i])
        dataframe['Anteckning'][i] = re.sub(' '+name+'\.', ' [PAD].', dataframe['Anteckning'][i])                
        dataframe['Anteckning'][i] = re.sub(' '+name+'\,', ' [PAD],', dataframe['Anteckning'][i])                
    return remove_y

names = ['Käkkir', 'Cecilia', 'Wexell', 'Neuront', 'Lisa', 'Hårsmar', 'Straumann', 'Alexandra', 'Gunilla', 'Laine', 'Västrik', 'Hannes', 'Olsson', 'Tissueve', 'Reet', 'Karlsson', 'Diagn', 'TissueLeve', 'Patolan', 'Kaoru', 'Oyaiz', 'PatEA', 'Fixop', 'BoneLevel', 'Anamn', 'Bengt', 'Tel', 'Stefan', 'Ahmed', 'Marianne', 'Flor', 'Vahedi', 'Christina', 'Backe', 'Anamnes', 'Ketac', 'Molar', 'Molarsalilo', 'Temp', 'Sofia', 'konstant', 'Mari', 'Louise', 'Odersjö', 'Helmbäck', 'Adam', 'manuell', 'Artling', 'Martin', 'Mvh', 'Tandl', 'Malin', 'Tina', 'Thakhrar', 'Alice', 'Per', 'HB', 'RN', 'Pia', 'Diana', 'Har', 'Göran', 'Kjeller', 'Johan', 'Malmström', 'Ketacfil', 'Anna', 'Lena', 'Nybesök', 'Kerstin', 'Persson', 'Pamol', 'Mattias', 'Anv', 'Hexiden', 'Wolak', 'Folkesson', 'Chlumsky', 'Katrin', 'Raunosal', 'ÅB', 'Remitte', 'Jenny', 'Petersson', 'Kerstn', 'Harriet', 'Ssk', 'Birgitta', 'dr', 'Leif', 'LindEA', 'Ringer', 'MadeleneEA', 'Madelene', 'Aron', 'Ascher', 'Yvonne', 'Anestesi', 'Id', 'MW', 'Björlins', 'Kir', 'Ling', 'Shiva', 'Raoufi', 'Carina', 'Vegard', 'Garsjö', 'AlimiTV', 'Sylte', 'Perk', 'Catharina', 'Jacobsson', 'Pedodonti', 'Reinfo', 'Hossein', 'Kashani', 'Larsson', 'Nina', 'Annika', 'Alexander', 'Dikran', 'Nellie', 'Jan', 'Erik', 'Johansson', 'Nikos', 'Merakos', 'Fogelin', 'Maria', 'Remissvar', 'Elisabeth', 'Kvissberg', 'Peter', 'Kofron', 'Or', 'Akutus', 'Machi', 'Papaioannou', 'Dr', 'Holger', 'Ann', 'Saer', 'Amir', 'Rehrmann', 'Peri', 'ANAMNES', 'Allm', 'Jacobsen', 'Marie', 'Lisbeth', 'Svalander', 'Christian', 'Schaefer', 'Fried', 'Eriksson', 'Glen', 'Levander', 'Op', 'Ingabritt', 'Lokala', 'Bone', 'Level', 'Lesnukhin', 'Kristina', 'Goldkuhl', 'Daganteck', 'Flors', 'St', 'Hill', 'Amlodipin', 'Wahlqvist', 'Kodein', 'konstantEA', 'Eva', 'Wridell', 'TissueLevel', 'Neck', 'Alveomg', 'Heikki', 'Lise', 'Lotte', 'Löfqvist', 'TVR', 'Lok', 'Patt', 'Klara', 'Madeleine', 'Smile', 'Vb500mg', 'Öhrnell', 'mr', 'Ink', 'Anne', 'Christine', 'Cederström', 'Leo', 'Victor', 'Bastante', 'Prolén', 'Cavalli', 'Björkman', 'Heracillin', 'Inger', 'Mona', 'Mikati', 'An', 'Kenny', 'Andersson', 'Waltbrand', 'Reto', 'Walterbrand', 'Janson', 'Ulrika', 'Gunnarsson', 'Patoka', 'Lars', 'Olof', 'Haider', 'Alveol', 'Remiss', 'Rosengren', 'KLAR', 'Manuel', 'O', 'Örnell', 'David', 'Öhman', 'Mats', 'Wallström', 'Roger', 'Akutu', 'Citodon', 'War', 'Ery', 'Max', 'Mo', 'Öhrnellkir', 'Dennis', 'Pal', 'Patrik', 'Vladimir', 'Susanne', 'Piezo', 'Sellgren', 'NacL', 'DO', 'John', 'Bratel', 'Ftv', 'Kulan', 'Von', 'Essen', 'Ulf', 'Rens', 'Rit', 'Lundh', 'Vidaza', 'Hege', 'Garelius', 'Westin', 'Dag', 'Sten', 'Mellqvist', 'Leszek', 'Ingela', 'Lissbrant', 'Hematolog', 'Pitack', 'Örtengren', 'Anki', 'Ceram', 'Mono', 'Nadja', 'Labus', 'Berit', 'BW', 'Kristoffer', 'Rehr', 'Mattis', 'Josephine', 'Luc', 'Åsa', 'Brånemark', 'Ti', 'Unite', 'Sanjiv', 'Kanagarajapn', 'Örnhall', 'Britse', 'Tomas', 'Nelsén', 'Adper', 'Scotchbon', 'Alberts', 'Albert', 'Morgan', 'Giovanni', 'Serino', 'Li', 'Lilliehöök', 'TH', 'Ivanoff', 'Vicryl', 'Linda', 'Annelie', 'Amanda', 'Klas', 'Ulenius', 'Diff', 'Eija', 'Gita', 'Gal', 'Blom', 'Karin', 'Camilla', 'Aneste', 'Björn', 'Fürst', 'Monica', 'Widman', 'Michelle', 'Ylva', 'von', 'Zweigberg', 'Askim', 'Clinomyn', 'Zeki', 'Yarbug', 'Corsodyl', 'Ring', 'Citod', 'Mir', 'Fuji', 'Nils', 'Buck1', 'Lokalanes', 'Sjöberg', 'Andreas', 'Schultz', 'Perkömmn', 'Rapid', 'Åker', 'Sunita', 'VP', 'Petra', 'Lindh', 'Marita', 'Thor', 'Malekzadeh', 'Karies', 'Lindemanns', 'Rythén', 'Johannes', 'Paulsson', 'Återbesök', 'Nobetec', 'Dalacin', 'Muale', 'Dalin', 'Sabel', 'Britt', 'Sofie', 'Hansén', 'Lipina', 'Samma', 'Elliot', 'Dak', 'Zoe', 'Gun', 'Lindquist', 'Anja', 'Claudia', 'Silva', 'Groz', 'Inj', 'Dimitrios', 'Karazisis', 'Gunnar', 'Stridh', 'GunnarEA', 'Ali', 'Strauman', 'Dahlström', 'Strömvall', 'Nasaluber', 'Jessica', 'Svanevik', 'Orion', 'Stina', 'Malm', 'Rhemans', 'Tommy', 'Saia', 'Vainalis', 'Tore', 'Däldborg', 'Kanagaraja', 'Jonasson', 'Razan', 'Rödbo', 'Sylvia', 'Niklas', 'Lindgren', 'Vita', 'Lovisa', 'Cassel', 'Cem', 'TTF', 'Arvidsson', 'Fort', 'Midazola', 'Mia', 'Carlsson', 'Hanna', 'Kostas', 'Bougas', 'Zoya', 'Holm', 'Eliquis', 'Anders', 'Franzén', 'Masoume', 'Zarandi', 'Arijan', 'Ismajli', 'Coltosol', 'Fuiji', 'Nathalie', 'Patricia', 'Ulla', 'Berglundh', 'Xgeva', 'Hofgren', 'Oss', 'Selma', 'Jakob', 'Sara', 'Brimstedt', 'Roya', 'Farzini', 'Molin', 'Frida', 'Hallgren', 'MeliorC', 'Michailidis', 'FTV', 'Horred', 'Torbjörn', 'Bursell', 'Patogen', 'Ev', 'Patuki', 'Waran', 'Cobolt', 'Crom', 'Collacone', 'Lundby', 'Nacl', 'Radhi', 'Diakins', 'Coesoft', 'Orudi', 'Wave', 'One', 'Gold', 'Berglund', 'Wallström800mg', 'Lennart', 'Ana', 'Menezes', 'Therese', 'Michael', 'Valentin', 'Ref', 'Helen', 'Olivia', 'Shahnaz', 'Ingrid', 'Row', 'Killey', 'Ser', 'GW', 'RosMarie', 'Axel', 'Lindberg', 'Morfin', 'Meda', 'Alex', 'AKUT', 'Willner', 'Andre', 'Thyg', 'JY', 'Golnar', 'Alveolit', 'Spoln', 'NaCpen', 'Tdl', 'Ulrike', 'Månstigen', 'Guldheden', 'Spol', 'NaCl', 'Vera', 'Asyl', 'Alkutus', 'Dick', 'Dockelberg', 'Ch', 'Dalgren', 'Olemyr', 'Ulrica', 'Inrem', 'Mirwais', 'Ahmadzai', 'Anamnessabr', 'Ap', 'Afrodite', 'Hudit', 'Hansson', 'Lindkvist', 'Rehrmans', 'Nora', 'Topas', 'Volodymyr', 'Alva', 'Emma', 'Kjell', 'Gadler', 'EDA', 'Börjesson', 'TetricFlow', 'RemittentTdl', 'Wallander', 'Krook', 'Orudis', 'Claesson', 'Dasmah', 'Gum', 'Paroex', 'Peiman', 'Flo', 'Samuelsson', 'NÄL', 'Fredrik', 'Widar', 'Tanja', 'Vis', 'Lorena', 'AnamnEA', 'Th', 'AF', 'Genrell', 'Nilsson', 'Evert', 'AP', 'Gråbo', 'Emina', 'Gvozdarxi', 'Gvozdar', 'Ak', 'Adel', 'Sundblad', 'JonasJohnson', 'Nyström', 'Alberto', 'Turri', 'Dahlén', 'Alveolar', 'Quick', 'Magnél', 'Patoe', 'Rönnqvist', 'Hawerman3950', 'Öhnell', 'Wertsén', 'Rem', 'Flytström', 'AA', 'Youse', 'Konstant', 'Bill', 'Alma', 'Ketogan', 'Ketoganod', 'Tramadol', 'Pateri', 'Olga', 'Jenssenmg', 'Adina', 'Popa', 'Ang', 'Lundqvist', 'Kristofer', 'Konopelk', 'Cerami', 'Gide', 'Ida', 'Kersin', 'Bo', 'Tor', 'Widbom', 'Christer', 'Kajsa', 'Elisab', 'Sundström', 'Nobote', 'Ewa', 'Eylertsen', 'Elin', 'Ringe', 'Mira', 'PatC', 'Jäwer', 'Anette', 'MOlar', 'Fredell', 'Mikael', 'Åby', 'Linnea', 'Patsody', 'Hultin', 'Gin', 'Matthias', 'Thomas', 'KS', 'Håkansson', 'Tetric', 'Evo', 'Nordberg', 'Parod', 'San', 'Cyrettage', 'Johanson', 'Sandra', 'Ekestubbe', 'Pietzo', 'IR', 'Elma', 'Misirlic', 'XA', 'Dental', 'Greger', 'mamm', 'Lalin', 'Ekelund', 'Elsa', 'Siva', 'Nargis', 'Nasiri', 'Domäng', 'Esters', 'hann', 'Jennifer', 'Ytter', 'Inga', 'Pelle', 'William', 'Persantin', 'Iliev', 'Rignell', 'Nordin', 'Aldell800mg', 'Zarah', 'Jimmy', 'Jour', 'Simon', 'Annes', 'Pedodont', 'Perkar', 'Ulvestam', 'Lassing', 'NaC', 'Eliquiz', 'Emanuelsson', 'Lundgren', 'Premed', 'Oraseal', 'Ren', 'Lina', 'Patsanti', 'Agneta', 'BoÅ', 'Lie', 'EvoCera', 'Lev', 'Straumannsu', 'Ghazaleh', 'Boroman', 'Volmefjord', 'Julia', 'Ing', 'Ash', 'Käk', 'Sofi', 'Brinkemo', 'Patric', 'Perma', 'Sharp', 'Daniel', 'EVA', 'Annelii', 'Malmqvist', 'Anas', 'Alexan', 'LO', 'Delvert', 'Bie', 'Holtz', 'Schondelmather', 'Patmgmg', 'Bod', 'Sixten', 'Alveo', 'Ivan', 'IT', 'Solbjörg', 'AAL', 'Dahlgren', 'Burgos', 'LE', 'LEL', 'Widmark', 'CAVE', 'Ket', 'Ralf', 'Caries', 'Ahmadza', 'Pernilla', 'Angelus', 'Rodica', 'Drag', 'Thy', 'Wahlberg', 'Plugger', 'Peg', 'Ragnar', 'Hanås', 'Patmg', 'Eliques', 'Arvidssons', 'Dela', 'Strömblad', 'Henrik', 'TEL', 'Zina', 'Kamal', 'Östgärd', 'Life', 'Patepraz', 'Angel', 'Nikolaos', 'Rigas', 'Carl', 'Stade', 'Patolärutskottetutskottet', 'Kihl', 'Åke', 'Marika', 'Toss', 'Zneugenol', 'Kåve', 'King', 'Son', 'Balverdi', 'Kariesbeh', 'Bodil', 'Rec', 'Lekholm', 'Zarb', 'DOEA', 'Angesjö', 'Heredit', 'Syberyjska', 'Mariana', 'Edvinsson', 'Decima', 'Gusmao', 'End', 'Amal', 'Jim', 'Axelsson', 'Melior', 'Borg', 'NaOC', 'Ibrahim', 'Gabrielson', 'Prem', 'Hosssein', 'KB', 'Par', 'Wallén', 'Nessrin', 'YEA', 'Lindqvist', 'Lofthag', 'Hansen', 'Tre', 'Trolle', 'Bengtsson', 'Sjögren', 'Ron', 'Mario', 'Malnar', 'Taki', 'Hatami', 'bergman', 'Johanna', 'Reinfo20mg', 'Lucas', 'Cecila', 'Saj', 'Stierner', 'Hussein', 'Barbro', 'Rahdi', 'Lith', 'Diskuss', 'Sadeer', 'Mahdi', 'Therése', 'Bettina', 'Pettersson', 'Claes', 'Löfström', 'Hewan', 'Tommaso', 'Padulaont', 'Goshgarian', 'Hijlkirurg', 'Maja', 'Kashaniaj', 'Helmi', 'Elisabet', 'Cecilila', 'Magda', 'Pateo', 'Hedda', 'Linn', 'Tveit', 'Pettersen', 'TempBond', 'Partner', 'Psykodonti', 'Barbara', 'Marek', 'Zoniuk', 'Romeborn', 'Joacim', 'Herculite', 'Mohamed', 'Lilly', 'Buccal', 'Judit', 'Kari', 'Sana', 'Mathilda', 'Ald', 'Un', 'Patff', 'Sally', 'Haux', 'Svalin', 'Sture', 'Cave', 'Rasmus', 'Samira', 'Blomqvist', 'Perikor', 'Vaselin', 'MTA', 'Sagitta', 'Corsodyldy', 'Erdal', 'Roland', 'Robert', 'Petersför', 'Alveolen', 'Andergrand', 'Brita', 'Holmström', 'Helena', 'Vivaden', 'Björk', 'Ansi', 'MH', 'Conny', 'Ingvar', 'Mirvais', 'Ma', 'Kmg', 'Nahida', 'Patlle', 'Alimi', 'Katja', 'Forsberg', 'Ahmad', 'Rasmusson', 'Rosann', 'Matilda', 'Örhnell', 'ASP', 'DanlosÄK', 'Céline', 'Ni', 'Helfer', 'Almehed', 'Egidiautskottetei', 'Tove', 'Hülya', 'Cevik', 'Ara', 'Käkkirurg', 'Oliwer', 'Tsk', 'EnarssonN2O', 'Ex', 'Edona', 'Kristian', 'Vestgärd', 'Angela', 'Terapi', 'Sendoline', 'Fil', 'Hassan', 'Alert', 'Enarsson', 'Frans', 'Wigenste', 'Suzan', 'Peñalo', 'Ingalill', 'Inganäs', 'Leon', 'Håkan', 'IngaliÄL', 'Ribbond', 'Revolution', 'Flow', 'Shakargi', 'Fredrick', 'Sender', 'Elnaz', 'Charlotte', 'Heijlontolog', 'Nyman', 'Molander', 'Diagnos', 'Engström', 'Oral', 'Stangenberg', 'Lennholm', 'Behnoosh', 'Mical', 'Tsega', 'Bertilsson', 'Silver', 'Liselott', 'Löfquist', 'Dahlin', 'Björns', 'Lövendal', 'manuelltan', 'LEF', 'Lund', 'LÖ', 'Carolina', 'Pattd', 'Preop', 'Marko', 'Ant', 'Lundin', 'Heil', 'Schander', 'Yasin', 'Mros', 'Anamnmg', 'Zahra', 'Salva', 'Göransson', 'Börresen', 'Excite', 'Vivapen', 'Lydia', 'Anneli', 'Nathan', 'Josefin', 'Åter', 'Jacqueline', 'Elzer', 'Fråg', 'Wid', 'Marigon', 'Wern', 'Aurell', 'Fransson', 'KSS', 'Oscar', 'Dakins', 'Gabriel', 'Livian', 'Carin', 'Wallgren', 'Helén', 'Svensson', 'V2b', 'Mårtensson', 'Helene', 'Hakee', 'Amelie', 'Kanagaraj', 'Petersén', 'Ragn', 'Göte', 'Alvedon', 'Ny', 'Tam', 'Stella', 'Gudnado´ttir', 'SG', 'II', 'Johnson', 'Ay', 'Lilian', 'Bråmark', 'Emilio', 'Siv', 'Knarrström', 'AliEAume', 'Premering', 'Lagerblad', 'Tallroth', 'On', 'Tallrothoka', 'Rangmar', 'Konsta', 'Nobel', 'Pat19', 'Fatane', 'Farjami', 'Oldin', 'Zachrisson', 'PASCAL', 'Vighage', 'Midazolam', 'Najda', 'Labusdy', 'Katarina', 'Sven', 'Edahlarina', 'Görans', 'Tissu', 'Friberg', 'Boden', 'Rönt', 'Rexon', 'Cina', 'Friedy', 'Avesta', 'ManuelEA', 'CharlotteEA', 'Stig', 'In', 'Aseel', 'Nashir', 'Sören', 'Dana', 'Rasoul', 'Vladomir', 'Ivarsson', 'Active', 'Gunnel', 'Azar', 'Joe', 'Gina', 'Hassel', 'Gustavsson', 'Prolia', 'Le', 'Dormicu', 'Asp', 'Alvar', 'Apelgren', 'Fjelddahl', 'Staveby', 'Fjeldahl', 'Cassandra', 'Bakiris', 'Opex', 'Thapper', 'Westberg', 'Leona', 'Malmberg', 'Ghaza', 'Boromand', 'Joneus', 'JK', 'SU', 'Bilgor', 'Heléna', 'Kasper', 'Henning', 'Suleiman', 'Remissemaby', 'Jävert', 'Hagberg', 'Tand', 'Lindehag', 'Coltoso', 'Alvaro', 'Campos', 'Augustsson', 'Patgin', 'Cle', 'Miranda', 'MSA', 'TDL', 'Gratian', 'Rosca', 'Bou', 'Rahal', 'Anita', 'Isackson', 'Arion', 'Henningsson', 'Lotta', 'Karazisi', 'Mina', 'Mehrpour', 'Ashkoor', 'Kumulin', 'Andri', 'Thorarinsson', 'Synthes', 'Leonardo', 'Antony', 'Ponjevic', 'Brateleloche', 'Rosbecker', 'AnnikaIPLA', 'Lundhal', 'Kronogård', 'Henric', 'KatjaEA', 'Kevin', 'louise', 'Marg', 'Block', 'Störner', 'sven', 'Wessam', 'Sanad', 'Kf', 'Ram', 'Szalaj', 'Jesper', 'Crist', 'Osseo', 'Speed', 'Selimovic', 'Welander', 'Ekwall', 'Samuel', 'Bagges', 'Pathak', 'Straumankir', 'Rozita', 'Ahlberg', 'Herliz', 'Hans', 'Herlitz', 'Petruson', 'Rane', 'FloantasEA', 'Uni', 'Lundberg', 'Majly', 'Hermansson', 'LindaTV', 'Bernt', 'Isberg', 'Dagante', 'Åland', 'Diffus', 'lena', 'Grankvist', 'Endo', 'Aspholm', 'Eyal', 'Mualem', 'Zelano', 'Oralt', 'Majvor', 'Goran', 'Mawlood', 'Arnemn', 'Bråne', 'Sheida', 'Zen', 'Xyl', 'Skallsjötan', 'Hak', 'Björlin', 'Vivian', 'Terese', 'Udevalla', 'Cyril', 'Pat3', 'Mun', 'Jelena', 'Jokic', 'Harrweg', 'Magnus', 'CurveEA', 'Magnusoka', 'Saga', 'RemittentT', 'Elgquist', 'Ebba', 'ERI', 'Franz', 'Remittent', 'Ismajl', 'Lil', 'Hasseus', 'Fjeldal', 'Annerud', 'Anform', 'Marielle', 'Hadil', 'Bondesson', 'Werlenius', 'Tronje', '[MASK]', 'Herman', 'Ehle', 'Danin', 'TERAPIPLAN', 'Fris', 'Liby', 'Röntgen', 'mannström', 'JohansonH', 'Lillieström', 'Rodriguezcit', 'Sulfa', 'Evy', 'Luna', 'Meta', 'Ferm', 'Sjögrenbynosol', 'Lindahl', 'Distansia', 'Dingle', 'Benny', 'Ytterby', 'KG', 'Lucy', 'Patricia150mg', 'Glyde', 'Götberg', 'Angelica', 'Emilsson', 'Maj', 'Hellström', 'Demens', 'Bio', 'Gidelen', 'Nordström', 'Parodoe', 'Damberg', 'Hasséus', 'Amberucveolärutskottetutskottet', 'Kenneth', 'Mabthera', 'Bendamustin', 'Olofe', 'KJ', 'Hemlin', 'Margit', 'Igelström', 'Lila', 'Regio', 'Melany', 'Elina', 'ssk', 'Felicia', 'Suska', 'Rondar', 'Matti', 'Koivuniemi', 'Jonika', 'Gjorgjieva', 'Bogren', 'Mörk', 'Savicutan', 'Kartous', 'AnnaMaj', 'Molande', 'Löf', 'Monika', 'El', 'Aaraj', 'Thorén', 'Lindome', 'TEPIPLA', 'Gabrielsson', 'Yngve', 'Submand', 'Unosson', 'Bartha', 'Pinex', 'Humelin', 'Krönle', 'vondt', 'Rehmann', 'Lenberg', 'Salwas', 'Berg', 'Adnan', 'Edta', 'Kla', 'Kent', 'Sköldborg', 'Bent', 'Neuro', 'Vinita', 'Stemol', 'Theo', 'Adna', 'Jonas', 'Sundberg', 'Pasjiaj', 'THEA', 'Ingbritt', 'Ekstedt', 'KristinaHenriksson', 'Thörnqvist', 'Kron', 'Trysberg', 'Estelle', 'Ilona', 'Henriksson', 'Boglar', 'Filip', 'Harn', 'Melander', 'Hugosson', 'Jarl', 'Ulceros', 'Col', 'Toner', 'Stenhardt', 'Vaslava', 'Warmark', 'Stenberg', 'Jaan', 'Sv', 'Marne', 'Landelius', 'Dagny', 'Blomgren', 'Ped', 'Tobias', 'Fagrell', 'Wlisabeth', 'Hägglin', 'Kollega', 'Fagerberg', 'Mohlin', 'Alv', 'Tål', 'Tailun', 'He', 'Folicke', 'Astma', 'Patden', 'Patpen', 'Pålsson', 'Trimetrop', 'Edvinson', 'Scotch', 'ScotchBond', 'Johnny', 'Patienten', 'Saris', 'Jonsson', 'Muhammed', 'Ahlqwist', 'Margareta', 'Haugen', 'Torun', 'Bågenholm', 'Chlum', 'Edita', 'Hagman', 'Sebastian', 'EM', 'Frimannson', 'Ramstedt', 'Artman', 'Winkelhoff', 'Telio', 'Ramhult', 'ANA', 'Karen', 'Nicholas', 'Pär', 'Brhane', 'Ehrndal', 'Hultgren', 'Hematologen', 'Dystervik', 'Ingemar', 'Veronica', 'CJ', 'Hullberg', 'Paroe', 'IvanoffÖ', 'Daganteckning', 'Padog', 'Karl', 'Eric', 'PatDen', 'Bankvall', 'Riva', 'Marcus', 'LEA', 'Isik', 'Manuell', 'ADIM', 'Jeanette', 'Sonya', 'Santos', 'Marcel', 'Billie', 'Flander', 'Anesteoka', 'Pascal', 'NaF', 'Ua', 'Isabella', 'LesnukhinO', 'PatF0', 'Margitji', 'Ekulf', 'Nas', 'Nasal', 'Odont', 'Bäckman', 'Ramone', 'Tepe', 'Ahlin', 'Gunnila', 'Pentti', 'Dur', 'Gen', 'Övl', 'Lill', 'Agnes', 'Wallmark', 'Vallgren', 'Ja', 'Rose', 'Kerr', 'Put', 'Ce', 'Moa', 'Folktand', 'Arthrotec', 'Pan', 'Vincent', 'Evelina', 'Duraphat', 'Ernesa', 'Naz', 'Mila', 'Bauer', 'Gunhild', 'Emrén', 'Charlotta', 'Widerström', 'Timote', 'Liljhagen', 'Rödström', 'ArtlingEA', 'Pre', 'Line', 'Wennerholm', 'Noushin', 'Nordlander', 'Patlien', 'Extraktion', 'Öv', 'Gerd', 'Riise', 'DIAGNOS', 'IL', 'Oraqix', 'Svanberg', 'Yasser', 'AliTidi', 'Anthmyr', 'Pihl', 'Törnblom', 'Dridol', 'Xeroden', 'Vega', 'Isabelle', 'Liv', 'Heggen', 'Adhesreme', 'Bendest', 'Hyg', 'Maxis', 'Thomasian', 'Sig', 'Anamnesgiv', 'Angle', 'Karolina', 'Berwald', 'Sanna', 'Årsjö', 'Saif', 'Al', 'Saidy', 'Harward', 'Filtek', 'Supreme', 'Frondelius', 'Aisomn', 'EvoFlow', 'Maisa', 'Walman', 'Rydén', 'Mircea', 'Albut', 'Christin', 'YEAR', 'Lukas', 'Janerot', 'Fourgel', 'nilsson', 'Rebecka', 'Gl', 'Plackers', 'Zekry', 'Emilia', 'Milding', 'Anela', 'Puty', 'André', 'Lagarto', 'JendebyTV', 'Kinna', 'Mimmi', 'Bettfys', 'Benhosh', 'Karazi', 'Bert', 'Kant', 'Durphat', 'Apodos', 'Ketak', 'Ulrikaph', 'Andrèa', 'Dalbergmn', 'PaganderTerapiplan', 'Ellinor', 'Nidal', 'Said', 'Min', 'Easy', 'Pick', 'Severinsson', 'Yusu', 'Yusuf', 'Re', 'SF', 'Emil', 'Mathias', 'Larborg', 'Jörgen', 'Viktor', 'Rim', 'Badawi', 'Inge', 'Flux', 'Mouth', 'Hesam', 'Sowlati', 'Manesh', 'Glide', 'Emine', 'Torstensson', 'Akutuin10', 'Maud', 'Wirén', 'Heidi', 'Margita', 'Rinse', 'Dyract', 'Benneth', 'Striver', 'Me', 'Collis', 'Curve', 'Rita', 'Pac', 'Topden', 'Jashar', 'Ek', 'IsbergEA', 'Gi', 'stensjö', 'EDTA', 'Bergman', 'Eivor', 'HjelmFolk', 'Tony', 'charlott', 'Pir', 'Thalin', 'vård', 'Mahvas', 'Mahvash', 'isb', 'Solo', 'Laila', 'Jansson', 'Kim', 'Lillemor', 'Ekenstråle', 'Kristenson', 'Bamse', 'Löfgren', 'Mazin', 'Amiira', 'Fernheim', 'Wojciech', 'Gierszewski', 'Patmn', 'Hassing', 'Rapide', 'Lokalanestes', 'Riskbedömning', 'Revus', 'Patzin', 'Marina', 'MP', 'Rosita', 'AndersTh', 'AnderssonF', 'Gingivits', 'Emelie', 'Sjölin', 'Billman', 'Michelsqu', 'Sertralin', 'Jonathan', 'Anton', 'Talal', 'Parwin', 'Holmgren', 'Zekiye', 'Na', 'Helisa', 'Parodmn', 'Pay', 'Etch', 'Rein', 'konstantoka', 'Alveoler', 'Antic', 'Frank', 'Manuelavdelning', 'Gente', 'Lidholms', 'Lidholm', 'Annamg', 'Hasan', 'Sarawood', 'Slh', 'Appelstrand', 'Emanuel', 'Fanny', 'Rosenlund', 'WallgrenF', 'Kvarford', 'Petronella', 'Norman', 'Jimmyod', 'RA', 'Joakim', 'Stefanos', 'Cheram', 'Ödman', 'Caesar', 'Tolk', 'Dari', 'Jagers', 'Daniels', 'Narges', 'Ber', 'Hy', 'Sama', 'Erika', 'Moberg', 'Stomvall', 'Ethel', 'Lindholm', 'Eliqu', 'Trybom', 'Rydbjer', 'Gustafsson', 'Pat1', 'Streman', 'Ina', 'Källström', 'Renhat', 'Viola', 'Hälsodekl', 'Gösta', 'Winqvist', 'DOL', 'Joel', 'Henelund', 'Klar', 'Pauline', 'Mirjam', 'Ringa', 'Mb', 'Crohn', 'Michels', 'Hök', 'Eyvor', 'Olle', 'Sarvenaz', 'Triage', 'Lennartsson', 'Jaber', 'sms', 'Mustafa', 'Skallsjö', 'Yasmin', 'Vlademir', 'Zarzioovi', 'KarinAn', 'Xarelto', 'Mild', 'Curettage', 'Bulle', 'ProExpert', 'Loka', 'Liza', 'Björndal', 'Viva', 'väninna', 'Solveig', 'IK', 'Carlgren', 'Göthberg', 'Jill', 'Patudi', 'Listerine', 'Dentan', 'Aya', 'Ayas', 'Ashko', 'NFR', 'Hiba', 'Charlott', 'Victoria', 'Jasmin', 'Mohammad', 'Annasex', 'Munasar', 'SMA', 'Karins', 'Tuggumi', 'Kost', 'Albin', 'Gliders', 'Wivi', 'Hygienis', 'Jennie', 'Robinson', 'Lisette', 'Hjelmgren', 'Anars', 'Pålssons', 'Kristin', 'DuraphatEA', 'Amber', 'Päivi', 'Alenfors', 'Elias', 'Zalin', 'Chatrine', 'Klovansky', 'Ellen', 'Excit', 'Dan', 'Gudrun', 'Pata5', 'Nannmark', 'Bjärne', 'Loise', 'Jana', 'Peggy', 'Widmans', 'Large', 'Soft', 'Junestål', 'Grönqvist', 'Karla', 'Norberg', 'Chi', 'Amelia', 'Vycr', 'Treiman', 'Capio', 'Ceceilia', 'Lagersköld', 'Britta', 'Hjördis', 'Trollér', 'Qvist', 'Nicolas', 'lie', 'Sagerbrink', 'Sundbye', 'Charlie', 'Zahran', 'Bahr', 'Valmire', 'Valmir', 'dagl', 'Britney', 'Ahlinder', 'Henrikv', 'Carola', 'WaveOne', 'Mathee', 'Oretad', 'Leila', 'Hjälpmedel', 'Sjögrens', 'Syndrom', 'Sabina', 'Hjälp', 'ListerineEA', 'Karlssonmn', 'Feras', 'Eid', 'Ria', 'XTE', 'Humira', 'Margaretha', 'Åhman', 'Andreasson', 'Westerhäll', 'Melisa', 'AKL', 'Ljungholm', 'Riitta', 'Pirjan', 'Greta', 'Camill', 'Panavia', 'Prime', 'Prel', 'Börje', 'Philippa', 'Baumgren', 'Jacob', 'Bohlin', 'Öl', 'Caroline', 'Fridh', 'Hamdi', 'Benyamin', 'manuellody', 'Jonatan', 'Kontroll', 'Tavegyl', 'Meijersjö', 'Unique', 'Ilida', 'Thomasiant', 'ESPE', 'Joanna', 'Warlin', 'Prof', 'Ove', 'Kazi', 'Triageod', 'Madde', 'Stana', 'Gyn', 'Linnmg', 'Andrei', 'Corsody', 'info', 'Christoffer', 'Darin', 'Pierre', 'Tutuml', 'Ridderheim', 'Borcic', 'Ribas', 'Jones', 'Vallmo', 'Patdyl', 'Harrys', 'Awara', 'Angspr', 'Annica', 'Adiels', 'Orvar', 'Rebecca', 'TorEA', 'Ceder', 'Birger', 'Molly', 'Nylund', 'Ringhald', 'Stv3A', 'Fru', 'Huntington', 'Polhem', 'Davidsson', 'sahlin', 'Sahlin10', 'Sub', 'OrudiEA', 'Åhrén', 'Corsdyl', 'Gel', 'Melker', 'Bedömning', 'Kai', 'Robin', 'Mirja', 'Kriisa', 'Rung', 'Thunfors', 'Nisse', 'Aktuellt', 'Wilma', 'Susanna', 'Jonzon', 'Jon', 'Peterson', 'Sophia', 'Bergbom', 'Belma', 'Marcelle', 'Bertilby', 'Kyl', 'Bett', 'Artrit', 'Williams', 'Boivie', 'Warvinge', 'Ragnhild', 'Björklund', 'Warving', 'Zagroz', 'Williamson', 'Ståhl', 'Gull', 'Cleaner', 'Anestesiolarutskottet', 'Hakan', 'Oganezova', 'Bankvalltas', 'Siw', 'Ljupco', 'Vevskoski', 'Melvin', 'Paulina', 'Ståhlucona', 'Lampe', 'Collins', 'Josef', 'Vass', 'Patisa', 'jan', 'Light', 'Cure', 'Cur', 'Mikkelsenmn', 'Utför', 'Rahil', 'konstantmn', 'TIArotepby', 'Lola', 'Quenselmn', 'Sose', 'Bjarne', 'Rolf', 'Mediciner', 'Odlogisk', 'Reyes', 'Lindström', 'Fatemeh', 'Babaie', 'Cathrine', 'Nylén', 'Andreasont', 'Shamso', 'Ginghat', 'Bäck', 'Gullbritt', 'Carnemar', 'Rosemary', 'Bartfay', 'Candida', 'Albican', 'Boguslaw', 'Ursula', 'Anamnesporges', 'Adrian', 'Es', 'Cornelia', 'Ind', 'Dagkl', 'Tormn', 'Herpes', 'Zooster', 'Olofsson', 'Halina', 'Hutchin', 'Bamyl', 'Luite', 'Omar', 'Shewakh', 'Desloratadine', 'Tilde', 'Linnéa', 'Richard', 'Susan', 'Joachim', 'Kicki', 'Annki', 'Nydén', 'Ossner', 'Arnell', 'Kallas', 'Jaredsson', 'Heimdahl', 'Muntorr', 'Bond', 'Bader', 'Svenne', 'Kimberley', 'Knapman', 'Evofl', 'HenningssonEA', 'Zekrya', 'File', 'Rite', 'EDTAsep', 'Forlax', 'Barmans', 'Norgren', 'Cit', 'Magnusson', 'Transpl', 'DIAGNOSER', 'Jendeby', 'Cucieman', 'Hörfält', 'helger', 'Denise', 'Wivianne', 'Wågsjöac']

print(names[:10])

c = 0
for n in names:
    remove_rows = remove_name(dataset_antibiotics, n)
    c+= len(remove_rows)
    
print('Names found: ',c)

### Converting strings to lower cases

In [ ]:
l = dataset_antibiotics.shape[0]
for i in range(l):
    line = dataset_antibiotics['Anteckning'][i]
    dataset_antibiotics.at[i, 'Anteckning'] = line.lower()
    

In [ ]:
def convert_pad(dataframe,name):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(name, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub('\['+name+'\]',' [PAD] ', dataframe['Anteckning'][i])
    return remove_y

remove_rows = convert_pad(dataset_antibiotics, 'pad')

### Final Dataset

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(dataset_antibiotics)
    dataset_antibiotics.to_csv('../data/dataset_no_recipe.csv', index = False)

### Create Text-to-Tex dataset

In [ ]:
dataset_antibiotics_seq = dataset_antibiotics.copy(deep=True)
dataset_antibiotics_seq['Anteckning'] = 'Antibiotics classification: ' + dataset_antibiotics_seq['Anteckning'].astype(str)
dataset_antibiotics_seq['Label'] = np.where(dataset_antibiotics_seq['Label'] == 1, 'Positive', 'Negative')

l = dataset_antibiotics_seq.shape[0]
for i in range(l):
    line = dataset_antibiotics_seq['Anteckning'][i]
    dataset_antibiotics_seq.at[i, 'Anteckning'] = line.lower()
    
remove_rows = convert_pad(dataset_antibiotics_seq, 'pad')

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(dataset_antibiotics_seq)
    dataset_antibiotics_seq.to_csv('../data/seq_dataset.csv', index = False)